# Simulation Results

## Policy Scenarios

We simulate six policy scenarios:

1. **Baseline (Profit Maximization)**: Stadium chooses optimal prices
2. **Current Observed Prices**: $80 tickets, $12.50 beer
3. **Price Ceiling ($7)**: Maximum beer price of $7
4. **Price Ceiling ($8)**: Maximum beer price of $8  
5. **Price Floor ($15)**: Minimum beer price of $15
6. **Beer Ban**: Zero beer sales
7. **Social Optimum**: Maximize social welfare including externalities

In [ ]:
import sys
sys.path.insert(0, '../')

from src.model import StadiumEconomicModel
from src.simulation import BeerPriceControlSimulator
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Initialize model with calibrated parameters
model = StadiumEconomicModel(
    capacity=46537,
    base_ticket_price=80.0,
    base_beer_price=12.5,
    ticket_elasticity=-0.625,
    beer_elasticity=-0.965,
    beer_cost=5.0,
    beer_excise_tax=0.074,
    beer_sales_tax_rate=0.08875,
    experience_degradation_cost=250.0
)

simulator = BeerPriceControlSimulator(model)

print("✓ Model initialized with calibrated parameters")

## Run All Scenarios

In [ ]:
# Run standard scenarios
results = simulator.run_all_scenarios(
    price_ceiling=8.0,
    price_floor=15.0,
    crime_cost_per_beer=2.5,
    health_cost_per_beer=1.5
)

# Add $7 price ceiling scenario
ceiling_7 = simulator.run_scenario(
    "Price Ceiling ($7)",
    beer_price_max=7.0,
    crime_cost_per_beer=2.5,
    health_cost_per_beer=1.5
)

# Combine results
results = pd.concat([results, pd.DataFrame([ceiling_7])], ignore_index=True)

# Display results
display_cols = [
    'scenario', 'consumer_beer_price', 'stadium_beer_price',
    'attendance', 'total_beers', 'profit', 'social_welfare',
    'total_tax_revenue', 'externality_cost'
]

results_display = results[display_cols].copy()
results_display = results_display.round(2)

results_display

## $7 Price Ceiling: Detailed Analysis

The $7 price ceiling represents a **binding constraint** (below profit-maximizing price of $12.85).

In [ ]:
# Extract $7 ceiling and baseline for comparison
baseline = results[results['scenario'] == 'Baseline (Profit Max)'].iloc[0]
ceiling_7_result = results[results['scenario'] == 'Price Ceiling ($7)'].iloc[0]
current = results[results['scenario'] == 'Current Observed Prices'].iloc[0]

print("=" * 80)
print("$7 PRICE CEILING ANALYSIS")
print("=" * 80)
print()

print("PRICES")
print(f"  Consumer pays:     ${ceiling_7_result['consumer_beer_price']:.2f} (was ${current['consumer_beer_price']:.2f})")
print(f"  Stadium receives:  ${ceiling_7_result['stadium_beer_price']:.2f} (was ${current['stadium_beer_price']:.2f})")
print(f"  Change for stadium: ${ceiling_7_result['stadium_beer_price'] - current['stadium_beer_price']:.2f}/beer")
print()

print("CONSUMPTION")
print(f"  Attendance:        {ceiling_7_result['attendance']:,.0f} (was {current['attendance']:,.0f})")
print(f"  Total beers:       {ceiling_7_result['total_beers']:,.0f} (was {current['total_beers']:,.0f})")
print(f"  Change:            {ceiling_7_result['total_beers'] - current['total_beers']:+,.0f} beers ({(ceiling_7_result['total_beers']/current['total_beers'] - 1)*100:+.1f}%)")
print()

print("STADIUM FINANCIALS")
print(f"  Profit:            ${ceiling_7_result['profit']:,.0f} (was ${current['profit']:,.0f})")
print(f"  Change:            ${ceiling_7_result['profit'] - current['profit']:+,.0f} ({(ceiling_7_result['profit']/current['profit'] - 1)*100:+.1f}%)")
print(f"  Per game:          ${ceiling_7_result['profit']:,.0f}")
print(f"  Per season (81):   ${ceiling_7_result['profit'] * 81:,.0f}")
print(f"  Annual loss:       ${(current['profit'] - ceiling_7_result['profit']) * 81:,.0f}")
print()

print("TAX REVENUE")
print(f"  Per game:          ${ceiling_7_result['total_tax_revenue']:,.0f} (was ${current['total_tax_revenue']:,.0f})")
print(f"  Change:            ${ceiling_7_result['total_tax_revenue'] - current['total_tax_revenue']:+,.0f}")
print(f"  Annual:            ${ceiling_7_result['total_tax_revenue'] * 81:,.0f}")
print()

print("SOCIAL WELFARE")
print(f"  Consumer surplus:  ${ceiling_7_result['consumer_surplus']:,.0f} (was ${current['consumer_surplus']:,.0f})")
print(f"  Producer surplus:  ${ceiling_7_result['producer_surplus']:,.0f} (was ${current['producer_surplus']:,.0f})")
print(f"  Externality cost:  ${ceiling_7_result['externality_cost']:,.0f} (was ${current['externality_cost']:,.0f})")
print(f"  Social welfare:    ${ceiling_7_result['social_welfare']:,.0f} (was ${current['social_welfare']:,.0f})")
print(f"  Change:            ${ceiling_7_result['social_welfare'] - current['social_welfare']:+,.0f}")
print()

print("=" * 80)

### Interpretation

The $7 price ceiling:

**Effects on Consumption:**
- Increases beer consumption (lower price → higher quantity)
- May increase attendance slightly (beer-ticket complementarity)

**Effects on Stadium:**
- **Reduces profit** significantly (lower margin despite higher volume)
- Reduces per-beer revenue from $11.41 to $6.35 (after taxes)
- Annual revenue loss: ~$1.8M

**Effects on Consumers:**
- **Increases consumer surplus** (lower prices)
- More affordable beer access
- May attract more price-sensitive fans

**Effects on Society:**
- **Higher externality costs** (more consumption → more crime/health impacts)
- Lower tax revenue (lower tax base)
- Net social welfare effect depends on consumer surplus gain vs externality increase

## Visual Comparison Across Scenarios

In [ ]:
# Create comparison visualizations
fig = go.Figure()

# Profit comparison
fig.add_trace(go.Bar(
    name='Stadium Profit',
    x=results['scenario'],
    y=results['profit'],
    marker_color='#003087'
))

fig.update_layout(
    title='Stadium Profit by Scenario',
    xaxis_title='Policy Scenario',
    yaxis_title='Profit per Game ($)',
    height=500,
    showlegend=False
)

fig.show()

In [ ]:
# Social welfare comparison
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=results['scenario'],
    y=results['consumer_surplus'],
    name='Consumer Surplus',
    marker_color='lightblue'
))

fig2.add_trace(go.Bar(
    x=results['scenario'],
    y=results['producer_surplus'],
    name='Producer Surplus',
    marker_color='lightgreen'
))

fig2.add_trace(go.Bar(
    x=results['scenario'],
    y=-results['externality_cost'],
    name='Externality Cost (negative)',
    marker_color='salmon'
))

fig2.update_layout(
    title='Welfare Components by Scenario',
    xaxis_title='Policy Scenario',
    yaxis_title='Value ($)',
    barmode='stack',
    height=600
)

fig2.show()

In [ ]:
# Beer consumption comparison
fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=results['scenario'],
    y=results['total_beers'],
    marker_color='#E4002B',
    text=results['total_beers'].round(0),
    textposition='outside'
))

fig3.update_layout(
    title='Total Beer Consumption by Scenario',
    xaxis_title='Policy Scenario',
    yaxis_title='Total Beers Sold',
    height=500,
    showlegend=False
)

fig3.show()

## Comparative Statics

Changes relative to current observed prices ($12.50 beer):

In [ ]:
# Calculate changes from baseline
changes = simulator.calculate_comparative_statics(results, baseline_scenario='Current Observed Prices')

# Display key changes
change_cols = [
    'scenario',
    'profit_change',
    'total_beers_change',
    'social_welfare_change',
    'externality_cost_change'
]

changes_display = changes[change_cols].copy()
changes_display = changes_display.round(0)

changes_display

## $7 vs $8 Price Ceiling Comparison

Comparing two different price ceiling levels:

In [ ]:
ceiling_comparison = results[results['scenario'].str.contains('Price Ceiling')].copy()

print("Price Ceiling Comparison:")
print()
for _, row in ceiling_comparison.iterrows():
    print(f"{row['scenario']}:")
    print(f"  Consumer price:    ${row['consumer_beer_price']:.2f}")
    print(f"  Stadium receives:  ${row['stadium_beer_price']:.2f}")
    print(f"  Total beers:       {row['total_beers']:,.0f}")
    print(f"  Stadium profit:    ${row['profit']:,.0f}")
    print(f"  Social welfare:    ${row['social_welfare']:,.0f}")
    print(f"  Externality cost:  ${row['externality_cost']:,.0f}")
    print()

# Calculate marginal effect of lowering ceiling from $8 to $7
ceiling_8 = results[results['scenario'] == 'Price Ceiling ($8.0)'].iloc[0]
ceiling_7_data = results[results['scenario'] == 'Price Ceiling ($7)'].iloc[0]

print("Marginal Effect of Lowering Ceiling from $8 to $7:")
print(f"  Beer consumption:  {ceiling_7_data['total_beers'] - ceiling_8['total_beers']:+,.0f} beers")
print(f"  Stadium profit:    ${ceiling_7_data['profit'] - ceiling_8['profit']:+,.0f}")
print(f"  Externality cost:  ${ceiling_7_data['externality_cost'] - ceiling_8['externality_cost']:+,.0f}")
print(f"  Social welfare:    ${ceiling_7_data['social_welfare'] - ceiling_8['social_welfare']:+,.0f}")

## Selection Effects Analysis

A key feature of the heterogeneous consumer model is that price policies change **who attends**, not just how many. The $7 ceiling increases drinker share because:
1. Ticket price rises, reducing attendance overall
2. Drinkers value cheap beer more than non-drinkers
3. Net effect: crowd composition shifts toward drinkers

In [ ]:
# Show selection effects if available
selection_cols = ['scenario', 'drinker_share', 'drinker_attendance', 'nondrinker_attendance', 
                  'drinker_beers_per_fan', 'nondrinker_beers_per_fan']

# Check which columns are available
available_cols = [c for c in selection_cols if c in results.columns]

if len(available_cols) > 1:
    print("SELECTION EFFECTS BY SCENARIO:")
    print()
    
    # Show drinker share changes
    if 'drinker_share' in results.columns:
        for _, row in results.iterrows():
            scenario = row['scenario']
            drinker_pct = row.get('drinker_share', 0.4) * 100
            print(f"  {scenario}:")
            print(f"    Drinker share: {drinker_pct:.1f}% (baseline 40%)")
            if 'drinker_beers_per_fan' in results.columns:
                print(f"    Drinker beers/fan: {row['drinker_beers_per_fan']:.2f}")
            print()
else:
    print("Selection effects metrics not available in this simulation run.")
    print("Run with heterogeneous model to see consumer type breakdown.")
    print()
    print("Conceptually, a $7 beer ceiling would:")
    print("  - Increase drinker share of attendance (they value cheap beer more)")
    print("  - Increase beers per drinker (lower price = higher quantity)")
    print("  - Combined effect: much higher total consumption per attendee")

## Summary Statistics

In [ ]:
summary = simulator.summary_statistics(results)

print("Key Scenarios:")
print(f"  Profit-maximizing:  {summary['profit_maximizing_scenario']}")
print(f"  Welfare-maximizing: {summary['welfare_maximizing_scenario']}")
print(f"  Lowest externality: {summary['lowest_externality_scenario']}")
print()

print("Attendance Range:")
print(f"  Mean: {summary['mean_attendance']:,.0f}")
print(f"  Std:  {summary['std_attendance']:,.0f}")
print(f"  Min:  {results['attendance'].min():,.0f}")
print(f"  Max:  {results['attendance'].max():,.0f}")
print()

print("Beer Consumption Range:")
print(f"  Mean: {summary['mean_total_beers']:,.0f}")
print(f"  Std:  {summary['std_total_beers']:,.0f}")
print(f"  Min:  {results['total_beers'].min():,.0f}")
print(f"  Max:  {results['total_beers'].max():,.0f}")

## Policy Implications

### Price Ceiling ($7)

**Winners:**
- Consumers (lower prices, higher surplus)
- Price-sensitive fans (increased access)

**Losers:**
- Stadium (lower profit)
- Government (lower tax revenue)
- Society (higher externality costs)

**Net Effect:** Depends on weight given to consumer surplus vs stadium profit vs externalities.

### Alternative: Pigouvian Tax

Instead of price ceiling, could add **$2.91/beer tax** to internalize external costs:
- Consumer price: $15.41
- Reduces consumption and externalities
- Raises revenue for affected communities
- More economically efficient than quantity controls